# Experiment 095: Ridge Regression for Better Generalization

**Hypothesis**: Simpler models may have a LOWER CV-LB intercept.

**The Problem**:
- All our models fall on the same CV-LB line: LB = 4.36 * CV + 0.052
- The intercept (0.052) > target (0.0347)
- This means CV optimization alone cannot reach the target

**Why Ridge Regression?**
1. Linear models are less prone to overfitting
2. Strong regularization prevents memorizing training distribution
3. May generalize better to unseen solvents
4. Simple features (Spange + ACS) capture chemistry without noise

**Features**: Spange (13) + ACS PCA (5) + Arrhenius kinetics (5) = 23 features

In [1]:
# Standard imports
import numpy as np
import pandas as pd
import torch
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler
import tqdm
import warnings
warnings.filterwarnings('ignore')

np.random.seed(42)
torch.set_default_dtype(torch.double)

print('Imports complete')

Imports complete


In [2]:
# Data loading functions
DATA_PATH = '/home/data'

INPUT_LABELS_NUMERIC = ["Residence Time", "Temperature"]
INPUT_LABELS_SINGLE_SOLVENT = ["Residence Time", "Temperature", "SOLVENT NAME"]
INPUT_LABELS_FULL_SOLVENT = ["Residence Time", "Temperature", "SOLVENT A NAME", "SOLVENT B NAME", "SolventB%"]

def load_data(name="full"):
    if name == "full":
        df = pd.read_csv(f'{DATA_PATH}/catechol_full_data_yields.csv')
        X = df[INPUT_LABELS_FULL_SOLVENT]
    else:
        df = pd.read_csv(f'{DATA_PATH}/catechol_single_solvent_yields.csv')
        X = df[INPUT_LABELS_SINGLE_SOLVENT]
    Y = df[["Product 2", "Product 3", "SM"]]
    return X, Y

def generate_leave_one_out_splits(X, Y):
    for solvent in sorted(X["SOLVENT NAME"].unique()):
        mask = X["SOLVENT NAME"] != solvent
        yield (X[mask], Y[mask]), (X[~mask], Y[~mask])

def generate_leave_one_ramp_out_splits(X, Y):
    ramps = X[["SOLVENT A NAME", "SOLVENT B NAME"]].drop_duplicates()
    for _, row in ramps.iterrows():
        mask = ~((X["SOLVENT A NAME"] == row["SOLVENT A NAME"]) & (X["SOLVENT B NAME"] == row["SOLVENT B NAME"]))
        yield (X[mask], Y[mask]), (X[~mask], Y[~mask])

print('Data loading functions defined')

Data loading functions defined


In [3]:
# Load feature lookups - ONLY Spange and ACS (no DRFP for simplicity)
SPANGE_DF = pd.read_csv(f'{DATA_PATH}/spange_descriptors_lookup.csv', index_col=0)
ACS_PCA_DF = pd.read_csv(f'{DATA_PATH}/acs_pca_descriptors_lookup.csv', index_col=0)

print(f'Spange: {SPANGE_DF.shape}, ACS PCA: {ACS_PCA_DF.shape}')
print(f'Total solvent features: {SPANGE_DF.shape[1] + ACS_PCA_DF.shape[1]}')

Spange: (26, 13), ACS PCA: (24, 5)
Total solvent features: 18


In [4]:
# Simple Featurizer with only Spange + ACS + Arrhenius kinetics
class SimpleFeaturizer:
    """Featurizer with only Spange + ACS + Arrhenius kinetics.
    
    Total features: 13 (Spange) + 5 (ACS) + 5 (kinetics) = 23 features
    """
    
    def __init__(self, mixed=False):
        self.mixed = mixed
        self.spange_df = SPANGE_DF
        self.acs_df = ACS_PCA_DF
        self.feats_dim = 2 + 3 + self.spange_df.shape[1] + self.acs_df.shape[1]  # 23 features

    def featurize(self, X, flip=False):
        X_vals = X[INPUT_LABELS_NUMERIC].values.astype(np.float64)
        temp_c = X_vals[:, 1:2]
        time_m = X_vals[:, 0:1]
        temp_k = temp_c + 273.15
        inv_temp = 1000.0 / temp_k
        log_time = np.log(time_m + 1e-6)
        interaction = inv_temp * log_time
        X_kinetic = np.hstack([X_vals, inv_temp, log_time, interaction])
        
        if self.mixed:
            A_spange = self.spange_df.loc[X["SOLVENT A NAME"]].values
            B_spange = self.spange_df.loc[X["SOLVENT B NAME"]].values
            A_acs = self.acs_df.loc[X["SOLVENT A NAME"]].values
            B_acs = self.acs_df.loc[X["SOLVENT B NAME"]].values
            
            r = X["SolventB%"].values.reshape(-1, 1)  # Already in [0, 1]
            
            if flip:
                r = 1 - r
                A_spange, B_spange = B_spange, A_spange
                A_acs, B_acs = B_acs, A_acs
            
            # Linear mixture
            X_spange = A_spange * (1 - r) + B_spange * r
            X_acs = A_acs * (1 - r) + B_acs * r
        else:
            X_spange = self.spange_df.loc[X["SOLVENT NAME"]].values
            X_acs = self.acs_df.loc[X["SOLVENT NAME"]].values
        
        return np.hstack([X_kinetic, X_spange, X_acs])

print(f'SimpleFeaturizer defined')
print(f'Feature dimension: {SimpleFeaturizer().feats_dim}')

SimpleFeaturizer defined
Feature dimension: 23


In [5]:
# Ridge Regression Model
class RidgeModel:
    """Ridge Regression with strong regularization for better generalization."""
    
    def __init__(self, data='single', alpha=10.0):
        self.data_mode = data
        self.mixed = (data == 'full')
        self.featurizer = SimpleFeaturizer(mixed=self.mixed)
        self.scaler = StandardScaler()
        self.alpha = alpha
        self.models = None
    
    def train_model(self, train_X, train_Y):
        # Get features
        X = self.featurizer.featurize(train_X)
        Y = train_Y.values
        
        # Data augmentation for mixtures
        if self.mixed:
            X_flip = self.featurizer.featurize(train_X, flip=True)
            X = np.vstack([X, X_flip])
            Y = np.vstack([Y, Y])
        
        # Scale features
        X_scaled = self.scaler.fit_transform(X)
        
        # Train Ridge models (one per target)
        self.models = []
        for i in range(3):
            model = Ridge(alpha=self.alpha, random_state=42)
            model.fit(X_scaled, Y[:, i])
            self.models.append(model)
    
    def predict(self, test_X):
        # Get features
        X = self.featurizer.featurize(test_X)
        
        # Scale features
        X_scaled = self.scaler.transform(X)
        
        # Predict
        preds = np.zeros((len(test_X), 3))
        for i in range(3):
            preds[:, i] = self.models[i].predict(X_scaled)
        
        # Clip to valid range
        preds = np.clip(preds, 0, 1)
        
        # Renormalize if sum > 1
        totals = preds.sum(axis=1, keepdims=True)
        divisor = np.maximum(totals, 1.0)
        preds = preds / divisor
        
        return torch.tensor(preds, dtype=torch.double)

print('RidgeModel defined with alpha=10.0')

RidgeModel defined with alpha=10.0


In [ ]:
########### DO NOT CHANGE ANYTHING IN THIS CELL OTHER THAN THE MODEL #################
########### THIS MUST BE THE THIRD LAST CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

import tqdm

X, Y = load_data("single_solvent")

split_generator = generate_leave_one_out_splits(X, Y)
all_predictions = []

for fold_idx, split in tqdm.tqdm(enumerate(split_generator)):
    (train_X, train_Y), (test_X, test_Y) = split

    model = RidgeModel(data='single', alpha=10.0)  # CHANGE THIS LINE ONLY
    model.train_model(train_X, train_Y)

    predictions = model.predict(test_X)  # Shape: [N, 3]

    # Move to CPU and convert to numpy
    predictions_np = predictions.detach().cpu().numpy()

    # Add metadata and flatten to long format
    for row_idx, row in enumerate(predictions_np):
        all_predictions.append({
            "task": 0,
            "fold": fold_idx,
            "row": row_idx,
            "target_1": row[0],
            "target_2": row[1],
            "target_3": row[2]
        })

# Save final submission
submission_single_solvent = pd.DataFrame(all_predictions)

########### DO NOT CHANGE ANYTHING IN THIS CELL OTHER THAN THE MODEL #################
########### THIS MUST BE THE THIRD LAST CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

In [ ]:
########### DO NOT CHANGE ANYTHING IN THIS CELL OTHER THAN THE MODEL #################
########### THIS MUST BE THE SECOND LAST CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

X, Y = load_data("full")

split_generator = generate_leave_one_ramp_out_splits(X, Y)
all_predictions = []

for fold_idx, split in tqdm.tqdm(enumerate(split_generator)):
    (train_X, train_Y), (test_X, test_Y) = split

    model = RidgeModel(data='full', alpha=10.0)  # CHANGE THIS LINE ONLY
    model.train_model(train_X, train_Y)

    predictions = model.predict(test_X)  # Shape: [N, 3]

    # Move to CPU and convert to numpy
    predictions_np = predictions.detach().cpu().numpy()

    # Add metadata and flatten to long format
    for row_idx, row in enumerate(predictions_np):
        all_predictions.append({
            "task": 1,
            "fold": fold_idx,
            "row": row_idx,
            "target_1": row[0],
            "target_2": row[1],
            "target_3": row[2]
        })

# Save final submission
submission_full_data = pd.DataFrame(all_predictions)

########### DO NOT CHANGE ANYTHING IN THIS CELL OTHER THAN THE MODEL #################
########### THIS MUST BE THE SECOND LAST CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

In [ ]:
########### DO NOT CHANGE ANYTHING IN THIS CELL #################
########### THIS MUST BE THE FINAL CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

submission = pd.concat([submission_single_solvent, submission_full_data])
submission = submission.reset_index()
submission.index.name = "id"
submission.to_csv("/home/submission/submission.csv", index=True)

########### DO NOT CHANGE ANYTHING IN THIS CELL #################
########### THIS MUST BE THE FINAL CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

In [ ]:
# Calculate CV score (for verification only - NOT part of submission)
X_single, Y_single = load_data("single_solvent")
X_full, Y_full = load_data("full")

# Get actuals in same order as predictions
actuals_single = []
for solvent in sorted(X_single["SOLVENT NAME"].unique()):
    mask = X_single["SOLVENT NAME"] == solvent
    actuals_single.append(Y_single[mask].values)
actuals_single = np.vstack(actuals_single)

actuals_full = []
ramps = X_full[["SOLVENT A NAME", "SOLVENT B NAME"]].drop_duplicates()
for _, row in ramps.iterrows():
    mask = (X_full["SOLVENT A NAME"] == row["SOLVENT A NAME"]) & (X_full["SOLVENT B NAME"] == row["SOLVENT B NAME"])
    actuals_full.append(Y_full[mask].values)
actuals_full = np.vstack(actuals_full)

# Get predictions
preds_single = submission_single_solvent[['target_1', 'target_2', 'target_3']].values
preds_full = submission_full_data[['target_1', 'target_2', 'target_3']].values

# Calculate MSE
mse_single = np.mean((actuals_single - preds_single) ** 2)
mse_full = np.mean((actuals_full - preds_full) ** 2)
n_single = len(actuals_single)
n_full = len(actuals_full)
overall_mse = (mse_single * n_single + mse_full * n_full) / (n_single + n_full)

print(f'\n=== CV SCORE VERIFICATION ===')
print(f'Single Solvent MSE: {mse_single:.6f} (n={n_single})')
print(f'Full Data MSE: {mse_full:.6f} (n={n_full})')
print(f'Overall MSE: {overall_mse:.6f}')
print(f'\nBest previous CV: 0.008092 (CatBoost+XGBoost)')
print(f'Best previous LB: 0.0877 (GP+MLP+LGBM)')
print(f'exp_030 baseline (GP+MLP+LGBM): CV 0.008298')
print(f'\nThis (Ridge Regression): CV {overall_mse:.6f}')

if overall_mse < 0.008092:
    improvement = (0.008092 - overall_mse) / 0.008092 * 100
    print(f'\n✓ IMPROVEMENT: {improvement:.2f}% better than best CV!')
elif overall_mse < 0.008298:
    improvement = (0.008298 - overall_mse) / 0.008298 * 100
    print(f'\n✓ IMPROVEMENT: {improvement:.2f}% better than exp_030!')
else:
    degradation = (overall_mse - 0.008298) / 0.008298 * 100
    print(f'\n✗ WORSE: {degradation:.2f}% worse than exp_030')

# Predicted LB based on CV-LB relationship
predicted_lb = 4.36 * overall_mse + 0.052
print(f'\nPredicted LB (based on CV-LB relationship): {predicted_lb:.4f}')
print(f'Best LB so far: 0.0877')